<a href="https://colab.research.google.com/github/Sapikzzz/AI-ASL/blob/main/Transferlearning-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import cv2
from sklearn.utils import shuffle
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import gc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_mobilenetv2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
import kagglehub
import seaborn as sns
import pandas as pd
import skimage
from skimage.transform import resize
from sklearn.metrics import classification_report, confusion_matrix
import os
from glob import glob
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import kagglehub
grassknoted_asl_alphabet_path = kagglehub.dataset_download('grassknoted/asl-alphabet')

train_path = os.path.join(grassknoted_asl_alphabet_path, '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train')
print('Data source import complete.')

Data source import complete.


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices()

tf.test.is_gpu_available()

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("GPU configured successfully")
    except RuntimeError as e:
        print("Error configuring GPU:", e)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Num GPUs Available:  1
GPU configured successfully


In [ ]:
target_size = (96, 96)
target_dims = (96, 96, 3)
n_classes = 29
val_frac = 0.1
batch_size = 256

data_augmentor = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    rescale=1./255,
    validation_split=val_frac
)

In [ ]:
train_gen = data_augmentor.flow_from_directory(train_path, target_size=target_size, batch_size=batch_size, shuffle=True, subset='training')
val_gen = data_augmentor.flow_from_directory(train_path, target_size=target_size, batch_size=batch_size, subset='validation')

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [ ]:
MobileNet_layers = MobileNetV2(weights='imagenet', include_top=False, input_shape=target_dims)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
for layer in MobileNet_layers.layers:
    layer.trainable = False

In [ ]:
MobileNet_model = Sequential([
    MobileNet_layers,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(n_classes, activation='softmax')
])
MobileNet_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_96             │ (None, 3, 3, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 29)             │        29,725 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,599,453 (13.73 MB)

 Trainable params: 1,341,469 (5.12 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
MobileNet_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
MobileNet_history = MobileNet_model.fit(train_gen, epochs=5, validation_data=val_gen)

Epoch 1/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 769s 3s/step - accuracy: 0.7470 - loss: 0.8309 - val_accuracy: 0.7907 - val_loss: 0.6558
Epoch 2/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 297s 970ms/step - accuracy: 0.8963 - loss: 0.3184 - val_accuracy: 0.8109 - val_loss: 0.6000
Epoch 3/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 304s 994ms/step - accuracy: 0.9203 - loss: 0.2398 - val_accuracy: 0.8305 - val_loss: 0.5257
Epoch 4/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 298s 972ms/step - accuracy: 0.9315 - loss: 0.2059 - val_accuracy: 0.8210 - val_loss: 0.5617
Epoch 5/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 297s 971ms/step - accuracy: 0.9362 - loss: 0.1919 - val_accuracy: 0.8222 - val_loss: 0.6025
